In [58]:
import numpy as np
from copy import deepcopy
import math

activities_to_classify = {
        'Get/replace items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open/close a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

activities_to_verbs = {
        'Get/replace items from refrigerator/cabinets/drawers': 'Get/Put',
        'Peel a cucumber': 'Slice',
        'Clear cutting board': 'Clean',
        'Slice a cucumber': 'Slice',
        'Peel a potato': 'Slice',
        'Slice a potato': 'Slice',
        'Slice bread': 'Slice',
        'Spread almond butter on a bread slice': 'Spread',
        'Spread jelly on a bread slice': 'Spread',
        'Open/close a jar of almond butter': 'Open/Close',
        'Pour water from a pitcher into a glass': 'Pour',
        'Clean a plate with a sponge': 'Clean',
        'Clean a plate with a towel': 'Clean',
        'Clean a pan with a sponge': 'Clean',
        'Clean a pan with a towel': 'Clean',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Get/Put',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Stack',
        'Stack on table: 3 each large/small plates, bowls': 'Stack',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load'
}

verbs = {
    'Get/Put': 0,
    'Load': 1,
    'Slice': 2,
    'Spread': 3,
    'Open/Close': 4,
    'Pour': 5,
    'Clean': 6,
    'Stack': 7,
}

def split_rows(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = 150 

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/30
            new_row['stop_timestamp'] = new_stop/30
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def create_annotations_file(timestamp_file, annotations_file_test, annotations_file_train):
    import pandas as pd

    video_fps = 30

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_timestamp'] = timestamps['start'] - start_timestamp
    timestamps['stop_timestamp'] = timestamps['stop'] - start_timestamp
    timestamps['start_frame'] = ((timestamps['start'] - start_timestamp) * video_fps).astype(int)
    timestamps['stop_frame'] = ((timestamps['stop'] - start_timestamp) * video_fps).astype(int)
    
    data_frame_train = pd.read_pickle(annotations_file_train)
    data_frame_train = data_frame_train[data_frame_train['file'] == 'S04_1.pkl']
    data_frame_train = data_frame_train.drop(['labels'], axis=1)
    data_frame_train['type'] = 'train'

    data_frame_test = pd.read_pickle(annotations_file_test)
    data_frame_test = data_frame_test[data_frame_test['file'] == 'S04_1.pkl']
    data_frame_test = data_frame_test.drop(['labels'], axis=1)
    data_frame_test['type'] = 'test'
    
    join_data_frame = pd.concat([data_frame_train, data_frame_test], ignore_index=True)

    result = pd.merge(timestamps, join_data_frame, on='index', how='inner')
    result.to_csv("JOINED.csv", index=False)
    
##--------------------------------------------------------------------------------------------------------------------
    result['uid'] = result['index']
    result['participant_id'] = 'S04'
    result['video_id'] = 'S04_01'
    result['verb'] = result['description_x']
    result['narration'] = result['description_x']
    result['verb_class'] = result['verb'].map(activities_to_classify)

    result = result[
        ['uid', 'participant_id', 'video_id', 'narration', 'start_timestamp', 'stop_timestamp', 'start_frame',
         'stop_frame', 'verb', 'verb_class', 'type']]
            
    result = result.sort_values(by='start_frame')
    

    result = split_rows(result)
    unique_random_uids = np.random.choice(range(len(result)), size=len(result), replace=False)
    result['uid'] = unique_random_uids
    
    reduced_labels = deepcopy(result)
    reduced_labels['verb'] = result['narration'].map(activities_to_verbs)
    reduced_labels['verb_class'] = result['narration'].map(activities_to_verbs).map(verbs).astype(int)
##--------------------------------------------------------------------------------------------------------------------

    test = result[result['type'] == 'test']
    test.drop('type', axis=1, inplace=True)
    test.to_csv("S04_test.csv", index=False)
    test.to_pickle("action_net_annotations/S04_test_complete.pkl")
    train = result[result['type'] == 'train']
    train.drop('type', axis=1, inplace=True)
    train.to_csv("S04_train.csv", index=False)
    train.to_pickle("action_net_annotations/S04_train_complete.pkl")
    
    test = reduced_labels[reduced_labels['type'] == 'test']
    test.drop('type', axis=1, inplace=True)
    test.to_csv("S04_test_reduced.csv", index=False)
    test.to_pickle("action_net_annotations/S04_test.pkl")
    train = reduced_labels[reduced_labels['type'] == 'train']
    train.drop('type', axis=1, inplace=True)
    train.to_csv("S04_train_reduced.csv", index=False)
    train.to_pickle("action_net_annotations/S04_train.pkl")

In [59]:
create_annotations_file('S04_1.pkl', 'action-net/ActionNet_test.pkl', 'action-net/ActionNet_train.pkl')

/var/folders/5s/flhrbc552q7bkv4fx1yszz1c0000gn/T/ipykernel_33512/4285880643.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('type', axis=1, inplace=True)
/var/folders/5s/flhrbc552q7bkv4fx1yszz1c0000gn/T/ipykernel_33512/4285880643.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('type', axis=1, inplace=True)
/var/folders/5s/flhrbc552q7bkv4fx1yszz1c0000gn/T/ipykernel_33512/4285880643.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [52]:
import pandas as pd

df = pd.read_pickle('action-net/ActionNet_test.pkl')

df = df[df['file'] == 'S04_1.pkl']

df

,index,file,description,labels
34,34,S04_1.pkl,Open/close a jar of almond butter,Open/Close
23,23,S04_1.pkl,Clear cutting board,Clear
26,26,S04_1.pkl,Spread almond butter on a bread slice,Spread
56,56,S04_1.pkl,"Set table: 3 each large/small plates, bowls, m...",Set
45,45,S04_1.pkl,Clean a plate with a towel,Clean
52,52,S04_1.pkl,Clean a pan with a towel,Clean
36,36,S04_1.pkl,Pour water from a pitcher into a glass,Pour
47,47,S04_1.pkl,Clean a plate with a towel,Clean


In [19]:
import pandas as pd

df = pd.read_pickle('action-net/ActionNet_train.pkl')

df = df[df['file'] == 'S04_1.pkl']

df

,index,file,description,labels
10,10,S04_1.pkl,Get/replace items from refrigerator/cabinets/d...,Get/Put
6,6,S04_1.pkl,Slice a cucumber,Slice
17,17,S04_1.pkl,Slice a potato,Slice
44,44,S04_1.pkl,Clean a plate with a sponge,Clean
37,37,S04_1.pkl,Pour water from a pitcher into a glass,Pour
27,27,S04_1.pkl,Spread almond butter on a bread slice,Spread
49,49,S04_1.pkl,Clean a pan with a sponge,Clean
33,33,S04_1.pkl,Open/close a jar of almond butter,Open/Close
29,29,S04_1.pkl,Spread jelly on a bread slice,Spread
24,24,S04_1.pkl,Get/replace items from refrigerator/cabinets/d...,Get/Put


In [18]:
import pandas as pd

df = pd.read_pickle('S04_1.pkl')

df

,description,start,stop,myo_left_timestamps,myo_left_readings,myo_right_timestamps,myo_right_readings
0,calibration,1.655239e+09,1.655240e+09,[],[],[],[]
1,Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09,"[1655239974.420555, 1655239974.432055, 1655239...","[[8, 7, 2, 2, 6, 5, 14, 2], [-4, 23, 19, 6, 16...","[1655239974.4195595, 1655239974.427058, 165523...","[[-2, -3, 2, 3, -8, -1, 1, -2], [3, 7, 8, -1, ..."
2,Peel a cucumber,1.655240e+09,1.655240e+09,"[1655240064.7444172, 1655240064.748419, 165524...","[[78, 57, 43, 7, 18, 18, 4, 16], [3, 0, 48, 6,...","[1655240064.7504191, 1655240064.76542, 1655240...","[[-18, 14, 26, -3, -15, -12, -26, 4], [-20, -2..."
3,Peel a cucumber,1.655240e+09,1.655240e+09,"[1655240130.5200577, 1655240130.531557, 165524...","[[-30, 20, 18, 1, 12, 23, 21, -10], [23, -15, ...","[1655240130.5190585, 1655240130.526558, 165524...","[[1, -21, 22, 14, 13, -24, -1, -12], [-58, -83..."
4,Peel a cucumber,1.655240e+09,1.655240e+09,"[1655240178.033217, 1655240178.037216, 1655240...","[[4, -19, -26, -63, -31, -36, -3, -1], [-30, -...","[1655240178.0472164, 1655240178.062216, 165524...","[[-4, 57, 49, 32, 34, 7, 7, 7], [-13, -80, -29..."
5,Clear cutting board,1.655240e+09,1.655240e+09,"[1655240238.6341186, 1655240238.645119, 165524...","[[-31, -13, 5, 33, 77, 20, 28, -20], [14, 16, ...","[1655240238.633119, 1655240238.636619, 1655240...","[[10, -9, -16, 0, -13, -2, -2, 2], [16, 15, 8,..."
6,Slice a cucumber,1.655240e+09,1.655240e+09,"[1655240291.109043, 1655240291.1125388, 165524...","[[7, -6, -2, -11, -22, -33, -17, 5], [-13, 3, ...","[1655240291.1150346, 1655240291.126035, 165524...","[[7, 13, 45, 14, -4, 1, 2, 4], [0, 2, -14, -10..."
7,Slice a cucumber,1.655240e+09,1.655240e+09,"[1655240357.383165, 1655240357.394165, 1655240...","[[-7, 12, 29, 4, -1, 18, -18, -24], [3, 14, 29...","[1655240357.378167, 1655240357.382166, 1655240...","[[3, 14, 9, 13, 11, 4, -11, -3], [-1, -12, -11..."
8,Slice a cucumber,1.655240e+09,1.655240e+09,"[1655240415.363077, 1655240415.3665743, 165524...","[[-14, -24, -14, -9, -8, -16, -47, -24], [-5, ...","[1655240415.3615725, 1655240415.365073, 165524...","[[-82, -25, -13, -28, -9, -75, 19, -1], [40, 1..."
9,Clear cutting board,1.655240e+09,1.655241e+09,"[1655240472.550571, 1655240472.5545695, 165524...","[[-8, 7, 20, 35, -11, 12, -11, 3], [-3, 17, 5,...","[1655240472.5575671, 1655240472.568567, 165524...","[[45, 31, 7, 14, 2, 3, -1, 12], [7, 6, -13, -1..."


In [25]:
import pandas as pd

df = pd.read_pickle('D4_test.pkl')
df = df.to_csv('D4_test.csv', index=False)

df

In [26]:
import pandas as pd

df = pd.read_pickle('D4_train.pkl')
df = df.to_csv('D4_train.csv', index=False)

df

In [31]:
import pandas as pd

df = pd.read_pickle('feature_extracted_D4_train.pkl')

df

{'features': [{'uid': 132,
   'video_name': 'P04_01',
   'features_RGB': array([[0.21563631, 0.17513026, 0.395112  , ..., 0.03039777, 0.47463724,
           0.40457177],
          [0.18741658, 0.13772085, 0.31015667, ..., 0.0065883 , 0.42935312,
           0.2470546 ],
          [0.16768369, 0.19179904, 0.31837642, ..., 0.00207361, 0.61217076,
           0.1821326 ],
          [0.17581138, 0.16059019, 0.29168928, ..., 0.01616938, 0.5257178 ,
           0.19435191],
          [0.23017485, 0.20737313, 0.33935523, ..., 0.02944645, 0.59196675,
           0.31951657]], dtype=float32)},
  {'uid': 2244,
   'video_name': 'P04_01',
   'features_RGB': array([[0.07277639, 0.36277327, 0.4509369 , ..., 0.00206367, 0.05169284,
           0.28292394],
          [0.08014341, 0.24846058, 0.28083214, ..., 0.00312716, 0.04174662,
           0.514032  ],
          [0.07627846, 0.21647987, 0.28985617, ..., 0.01863924, 0.08325351,
           0.36027342],
          [0.03635548, 0.3698099 , 0.3633184 , ..., 0

In [33]:
import pandas as pd

df = pd.read_pickle('ActionNet_train.pkl')

df

,index,file,description,labels
40,40,S08_1.pkl,Spread jelly on a bread slice,Spread
10,10,S04_1.pkl,Get/replace items from refrigerator/cabinets/d...,Get/Put
6,6,S03_1.pkl,Clear cutting board,Clear
24,24,S08_1.pkl,Clear cutting board,Clear
6,6,S04_1.pkl,Slice a cucumber,Slice
...,...,...,...,...
49,49,S05_2.pkl,Clean a pan with a sponge,Clean
56,56,S09_2.pkl,Clean a plate with a sponge,Clean
3,3,S03_2.pkl,Spread almond butter on a bread slice,Spread
6,6,S02_2.pkl,Slice a cucumber,Slice
